In [ ]:
import pandas as pd
import numpy as np
# import torch
from sklearn import preprocessing

seed = 42
FILENAME = "train_dataset.csv"

In [ ]:
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

y = df["type"]
df = df.drop(columns=["type"])

oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Distribution of each class
counts = np.bincount(y)
percentages = counts / counts.sum() * 100
plt.figure(figsize=(12,5))
plt.grid(True, axis='y', linestyle=(0,(5,10)))
plt.bar(le.classes_, percentages)
for i, percentage in enumerate(percentages):
    plt.text(i, percentage+0.5, f'{percentage:.2f}%', ha='center')

In [ ]:
# Boxplot of each feature
plt.figure(figsize=(12,5))
plt.boxplot(X, tick_labels=range(X.shape[1]), orient='v')

In [ ]:
# Variance for each feature
plt.figure(figsize=(12,5))
plt.grid(True, axis='y', linestyle=(0,(5,10)))
plt.bar(range(X.shape[1]), np.var(X, axis=0))

In [ ]:
# Correlation matrix
corr_matrix = np.corrcoef(X, rowvar=False)

plt.figure(figsize=(12,5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', xticklabels=range(X.shape[1]))